In [ ]:
# =============================
# Data Extraction Notebook
# Project: Energy Prices in Illinois
# Milestone 1
# =============================

In [1]:
# Import required libraries
import pandas as pd
import requests
from io import StringIO
import pickle
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# -----------------------------
# 1. Define Save Paths
# -----------------------------
BASE_PATH = "/content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois"
RAW_PATH = "/content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/raw"
PROC_PATH = "/content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/processed"
os.makedirs(RAW_PATH, exist_ok=True)
os.makedirs(PROC_PATH, exist_ok=True)

In [4]:
# -----------------------------
# 2. Define Data Sources
# -----------------------------
ACTUAL_CSV    = os.path.join(RAW_PATH, "actual_energy_price.csv")
DAY_AHEAD_CSV = os.path.join(RAW_PATH, "day_ahead_energy_price.csv")

In [5]:
# -------------------------------
# 3. Helper Function (read CSV)
# -------------------------------
def load_energy_csv(path, name="data"):
    print(f"\nReading {name} from {path} ...")
    df = pd.read_csv(path)

    cols_lower = {c.lower().strip(): c for c in df.columns}
    date_col  = cols_lower.get("date") or cols_lower.get("datetime") or list(df.columns)[0]
    price_col = cols_lower.get("lmp")  or cols_lower.get("price")
    hub_col   = cols_lower.get("hub")  or cols_lower.get("node") or cols_lower.get("location")

    use_cols = [date_col, price_col, hub_col]
    df = df[use_cols].rename(columns={
        date_col:  "date",
        price_col: "price",
        hub_col:   "hub"
    })

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date", "price"]).sort_values("date").reset_index(drop=True)

    print("Columns:", list(df.columns))
    print(df.head(3))
    return df


In [6]:
# -------------------------------
# 4. Load Datasets (from CSVs)
# -------------------------------
df_actual    = load_energy_csv(ACTUAL_CSV,    "actual_energy_price.csv")
df_day_ahead = load_energy_csv(DAY_AHEAD_CSV, "day_ahead_energy_price.csv")


Reading actual_energy_price.csv from /content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/raw/actual_energy_price.csv ...


/tmp/ipython-input-3777422313.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


Columns: ['date', 'price', 'hub']
        date  price           hub
0 2020-10-19  19.59  ARKANSAS.HUB
1 2020-10-19  19.80  ILLINOIS.HUB
2 2020-10-19  20.11   INDIANA.HUB

Reading day_ahead_energy_price.csv from /content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/raw/day_ahead_energy_price.csv ...


/tmp/ipython-input-3777422313.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


Columns: ['date', 'price', 'hub']
        date  price           hub
0 2020-10-19  20.50  ARKANSAS.HUB
1 2020-10-19  20.62  ILLINOIS.HUB
2 2020-10-19  20.98   INDIANA.HUB


In [7]:
#Hourly Pricing API data

import pandas as pd
import requests
from datetime import datetime, timedelta

def fetch_comed_5min(start_dt, end_dt):
    """Fetch 5-min prices between start_dt and end_dt (both naive datetimes in local time)."""
    fmt = "%Y%m%d%H%M"
    datestart = start_dt.strftime(fmt)
    dateend   = end_dt.strftime(fmt)
    url = (
        "https://hourlypricing.comed.com/api"
        f"?type=5minutefeed&datestart={datestart}&dateend={dateend}&format=json"
    )
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    if not data:
        return pd.DataFrame(columns=["date", "price", "hub"])
    df = pd.DataFrame(data)
    df["millisUTC"] = df["millisUTC"].astype("int64")
    # Convert millisUTC → datetime in Central Time
    df["date"] = (
        pd.to_datetime(df["millisUTC"], unit="ms", utc=True)
          .dt.tz_convert("America/Chicago")
          .dt.tz_localize(None)
    )
    df["price"] = df["price"].astype(float)
    df["hub"] = "COMED.5MIN"
    return df[["date", "price", "hub"]]

# Example: pull 1 month of data in 7-day chunks
start = datetime(2024, 10, 1)
end   = datetime(2024, 10, 31)

dfs = []
chunk_start = start
while chunk_start <= end:
    chunk_end = min(chunk_start + timedelta(days=7), end)
    print("Fetching", chunk_start, "→", chunk_end)
    df_chunk = fetch_comed_5min(chunk_start, chunk_end)
    dfs.append(df_chunk)
    chunk_start = chunk_end + timedelta(minutes=5)
df_comed = pd.concat(dfs, ignore_index=True).sort_values("date")
df_comed.head(), df_comed.tail()


Fetching 2024-10-01 00:00:00 → 2024-10-08 00:00:00
Fetching 2024-10-08 00:05:00 → 2024-10-15 00:05:00
Fetching 2024-10-15 00:10:00 → 2024-10-22 00:10:00
Fetching 2024-10-22 00:15:00 → 2024-10-29 00:15:00
Fetching 2024-10-29 00:20:00 → 2024-10-31 00:00:00


(                    date  price         hub
 2011 2024-10-01 00:00:00    1.8  COMED.5MIN
 2010 2024-10-01 00:05:00    1.7  COMED.5MIN
 2009 2024-10-01 00:10:00    2.0  COMED.5MIN
 2008 2024-10-01 00:15:00    1.9  COMED.5MIN
 2007 2024-10-01 00:20:00    4.6  COMED.5MIN,
                     date  price         hub
 8056 2024-10-30 23:40:00    1.5  COMED.5MIN
 8055 2024-10-30 23:45:00    1.2  COMED.5MIN
 8054 2024-10-30 23:50:00    1.2  COMED.5MIN
 8053 2024-10-30 23:55:00    1.2  COMED.5MIN
 8052 2024-10-31 00:00:00    1.2  COMED.5MIN)

In [8]:
# -----------------------------
# 5. Merge & Add Features
# -----------------------------
if not df_actual.empty and not df_day_ahead.empty:
    df = pd.merge(
        df_actual, df_day_ahead,
        on="date", how="inner", suffixes=("_actual", "_dayahead")
    )

    # Calendar features
    df["day_of_week"] = df["date"].dt.day_name()
    df["hour_of_day"] = df["date"].dt.hour
    df["month"] = df["date"].dt.month

    print("✅ Merged dataset:", df.shape)
else:
    print("❌ One of the datasets is empty.")
    df = pd.DataFrame()

✅ Merged dataset: (2028368, 8)


In [9]:
df_miso_long = pd.concat([
    df_actual.assign(source="MISO_ACTUAL"),
    df_day_ahead.assign(source="MISO_DAYAHEAD")
], ignore_index=True)

# Add COMED with a source tag
df_comed_long = df_comed.assign(source="COMED_5MIN")

# Combine all
df_all = pd.concat([df_miso_long, df_comed_long], ignore_index=True)

# Sort and reset index
df_all = df_all.sort_values("date").reset_index(drop=True)

print("Final combined dataset shape:", df_all.shape)
print(df_all.head())

Final combined dataset shape: (609683, 4)
        date  price            hub         source
0 2020-10-19  19.59   ARKANSAS.HUB    MISO_ACTUAL
1 2020-10-19  20.98    INDIANA.HUB  MISO_DAYAHEAD
2 2020-10-19  20.26  LOUISIANA.HUB  MISO_DAYAHEAD
3 2020-10-19  20.98   MICHIGAN.HUB  MISO_DAYAHEAD
4 2020-10-19  21.25       MINN.HUB  MISO_DAYAHEAD


In [10]:
# -----------------------------
# 6. Debug & Working Datasets
# -----------------------------
df_debug = df_all.head(500)      # small subset for quick testing
df_working = df_all.copy()       # full dataset

In [11]:
# -----------------------------
# 7. Save Processed Data
# -----------------------------
debug_pickle = os.path.join(PROC_PATH, "energy_debug.pkl")
working_pickle = os.path.join(PROC_PATH, "energy_working.pkl")

df_debug.to_pickle(debug_pickle)
df_working.to_pickle(working_pickle)

print(f"💾 Saved processed debug -> {debug_pickle}")
print(f"💾 Saved processed working -> {working_pickle}")

💾 Saved processed debug -> /content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/processed/energy_debug.pkl
💾 Saved processed working -> /content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/processed/energy_working.pkl
